# Struktur folder

Berikut adalah struktur folder source-code yang terupload di Github

In [ ]:
DAA_Instances/
├─ data/                      # berisi instance JSON (15 proyek)
├─ run.py                     # skrip eksekusi cepat
├─ generate_instances.py      # pembuat instance tambahan (per kelompok/seed)
└─ README.md

# 1) Skrip eksekusi: run.py

In [ ]:
# DAA_Instances/run.py
import json, argparse, time
from pathlib import Path
from typing import Any

# -------------------- ALGORITMA CONTOH --------------------
# Contoh implementasi penuh hanya untuk proyek "sorting".
# Proyek lain tetap memakai format instance yang sudah konsisten
# (isi algo_A/algo_B sendiri sesuai topik kelompok).
def algo_A(instance: Any, project: str) -> Any:
    if project == "sorting":
        # Insertion Sort (contoh Algoritma A)
        arr = list(instance["array"])
        for i in range(1, len(arr)):
            key = arr[i]; j = i - 1
            while j >= 0 and arr[j] > key:
                arr[j+1] = arr[j]; j -= 1
            arr[j+1] = key
        return arr
    raise NotImplementedError(f"algo_A not implemented for project={project}")

def algo_B(instance: Any, project: str) -> Any:
    if project == "sorting":
        # Merge Sort (contoh Algoritma B)
        def merge_sort(a):
            if len(a) <= 1: return a
            m = len(a)//2
            L = merge_sort(a[:m]); R = merge_sort(a[m:])
            i = j = 0; out = []
            while i < len(L) and j < len(R):
                if L[i] <= R[j]: out.append(L[i]); i += 1
                else: out.append(R[j]); j += 1
            out.extend(L[i:]); out.extend(R[j:])
            return out
        return merge_sort(list(instance["array"]))
    raise NotImplementedError(f"algo_B not implemented for project={project}")

# -------------------- EVALUATOR (ubah per proyek) --------------------
def evaluate(instance: Any, output: Any, project: str) -> float:
    """Kembalikan metrik kualitas (0 = sempurna).
       Lengkapi evaluator sesuai proyek (gap, biaya, kelayakan, dsb.)."""
    if project == "sorting":
        return 0.0 if all(output[i] <= output[i+1] for i in range(len(output)-1)) else 1.0
    return 0.0

def main():
    p = argparse.ArgumentParser()
    p.add_argument('--instance', required=True, help='Path ke berkas JSON di folder data/')
    p.add_argument('--algo', choices=['A','B'], default='A')
    args = p.parse_args()

    inst = json.load(open(args.instance, 'r', encoding='utf-8'))
    project = inst.get("project", "unknown")

    t0 = time.perf_counter()
    out = algo_A(inst, project) if args.algo == 'A' else algo_B(inst, project)
    dt = (time.perf_counter() - t0) * 1000.0
    gap = evaluate(inst, out, project)

    print(f"Project={project}  Algo={args.algo}  Time_ms={dt:.2f}  Gap={gap:.4f}")

if __name__ == '__main__':
    main()


# 2) Generator anti-overlap: generate_instances.py
* Tujuan: membuat varian instance per kelompok (1…15) → tidak overlap.
* Contoh di bawah memvariasikan sorting; Anda bisa meniru pola yang sama untuk proyek lain.

In [ ]:
# DAA_Instances/generate_instances.py
import json, random
from pathlib import Path

BASE = Path(__file__).resolve().parent / 'data'
BASE.mkdir(exist_ok=True)

def save(name, obj):
    p = BASE / name
    with open(p, 'w', encoding='utf-8') as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    return p

def make_sorting_near_sorted(n=2000, swap_every=50, seed=42):
    arr = list(range(n))
    rnd = random.Random(seed)
    # near-sorted: tukar kecil tiap interval
    for i in range(0, n, swap_every):
        if i+1 < n:
            arr[i], arr[i+1] = arr[i+1], arr[i]
    return arr

def main():
    for g in range(1, 16):
        arr = make_sorting_near_sorted(n=1500 + 20*g, swap_every=40, seed=100+g)
        save(f'sorting_near_sorted_G{g:02d}.json',
             {"project":"sorting","array":arr})
    print("OK: 15 varian sorting dibuat. Tiru polanya untuk proyek lain agar instance unik per kelompok.")

if __name__ == '__main__':
    main()


# 3) Contoh 15 instance (ringkas) — simpan di DAA_Instances/data/
* Semua JSON memuat kunci `project` sebagai pengarah evaluator/loader.
* Bisa menambah field lain sesuai kebutuhan tiap kelompok.

## Contoh Project

> Semua JSON memuat kunci "project" sebagai pengarah evaluator/loader.
> Anda bisa menambah field lain sesuai kebutuhan tiap kelompok.

### Project Penjadwalan (Interval Schedulling) - weekly_schedulling_ftisd_mingguA.json

In [ ]:
{
  "project": "scheduling_activity_selection",
  "description": "FTISD room scheduling week A; times in minutes since day start",
  "activities": [
    {"id":"C101","start":480,"finish":540,"weight":3},
    {"id":"C102","start":510,"finish":600,"weight":2},
    {"id":"C103","start":560,"finish":620,"weight":4},
    {"id":"C104","start":610,"finish":700,"weight":3},
    {"id":"C105","start":650,"finish":720,"weight":5},
    {"id":"C106","start":730,"finish":800,"weight":2}
  ]
}


# 4) README.md (ringkasan)

In [ ]:
DAA Instance Package (Proyek Schedulling)
- data/: instance JSON
- run.py: eksekusi cepat -> contoh implementasi hanya untuk 'sorting'
- generate_instances.py: contoh generator varian per kelompok (1–15)

Contoh pakai:
python run.py --instance data/sorting_near_sorted.json --algo A
python run.py --instance data/sorting_near_sorted.json --algo B


## Cara pakai cepat

1. Buat folder `DAA_Instances/`, simpan `run.py`, `generate_instances.py`, dan semua JSON di atas ke subfolder `data/`.
2. Coba jalankan:

In [ ]:
python run.py --instance data/sorting_near_sorted.json --algo A
python run.py --instance data/sorting_near_sorted.json --algo B

Akan membuat `sorting_near_sorted_G01..G15.json`. Tiru polanya untuk proyek lain (mis. knapsack/tsp: variasikan seed/ukuran/kepadatan/kapasitas).